In [1]:
import torch
from torch import nn as nn
import torchvision
from torch.utils.data import DataLoader, dataset
from torchvision.transforms import AutoAugment, Normalize, Compose, ToTensor
import torch.nn.functional as F
from torchmetrics import Accuracy
import numpy as np

In [2]:
class accuracy:
    def __init__(self):
        pass
    def update(self, x,y):
        pass
    def reset(self):
        pass

In [3]:
def __normalize__(x):
    return x/255.
normalize = torch.jit.script(__normalize__)

In [4]:
from torchvision.datasets import CIFAR100

In [5]:
data = CIFAR100("./data", train = True, download = True, transform = Compose([AutoAugment(),ToTensor(), normalize]))
test_data = CIFAR100("./data", train = False, download = True, transform = Compose([AutoAugment(),ToTensor(),  normalize]))

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_set = DataLoader(data, 128, shuffle = True, num_workers=2)
test_set = DataLoader(test_data, 128, shuffle = False, num_workers=1)

In [7]:
class VGG5(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq_1 = nn.Sequential(nn.Conv2d(3, 64, 2,2), nn.GELU(),                                                                     
                                   nn.BatchNorm2d(64))
        self.seq_2 = nn.Sequential(nn.Conv2d(64, 128, 2,1), nn.GELU(),
                                   nn.Conv2d(128, 128, 2,1), nn.GELU(), 
                                   nn.Conv2d(128, 128, 2,1), nn.GELU(), 
                                   nn.MaxPool2d(2,2, ), 
                                   nn.BatchNorm2d(128))
        self.seq_3 = nn.Sequential(nn.Conv2d(128, 256, 2,1, padding = "same"), nn.GELU(),
                                   nn.Conv2d(256, 256, 2,1, padding = "same"), nn.GELU(), 
                                   nn.Conv2d(256, 256, 2,1, padding = "same"), nn.GELU(), 
                                   nn.MaxPool2d(2,2, padding = 1), 
                                   nn.BatchNorm2d(256, ))
        self.seq_4 = nn.Sequential(nn.Conv2d(256, 512, 2,1, padding = "same"), nn.GELU(),
                                   nn.Conv2d(512, 512, 2,1, padding = "same"), nn.GELU(), 
                                   nn.Conv2d(512, 512, 2,1, padding = "same"), nn.GELU(), 
                                   nn.MaxPool2d(2,2, padding = 1), 
                                   nn.BatchNorm2d(512, ))
        self.seq_5 = nn.Sequential(nn.Conv2d(512, 1024, 2,1, padding = "same"), nn.GELU(),
                                   nn.MaxPool2d(2,2), 
                                   nn.BatchNorm2d(512, ))
        
        self.f = nn.Flatten()
        self.dense = nn.Linear(4608, 100)
    def forward(self, x):
        x = self.seq_1(x)
        x = self.seq_2(x)
        x = self.seq_3(x)
        x = self.seq_4(x)
        x = self.f(x)
        x = self.dense(x)
        return x

In [8]:
model_ = VGG5()
model = torch.nn.DataParallel(model_)
model.to(0)


DataParallel(
  (module): VGG5(
    (seq_1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(2, 2), stride=(2, 2))
      (1): GELU(approximate='none')
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
      (1): GELU(approximate='none')
      (2): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1))
      (3): GELU(approximate='none')
      (4): Conv2d(128, 128, kernel_size=(2, 2), stride=(1, 1))
      (5): GELU(approximate='none')
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (seq_3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (1): GELU(approximate='none')
      (2): Conv2d(256, 256, kernel_size=(2, 2), stride=(1, 1), padding=same)
      (3): GELU(approximate='n

In [9]:
model(torch.randn(1,3,32,32)).shape

/home/sahmaran/anaconda3/lib/python3.9/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv2d(input, weight, bias, self.stride,


torch.Size([1, 100])

In [10]:
def loss(x,y):
    return nn.CrossEntropyLoss()(model(x),y)

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)

In [ ]:
t = 0
while True:
    L = []
    loss__ = []
    model.train()

    for i, batch in enumerate(train_set):
        x, y = batch[0].cuda(), batch[1].cuda()
    
        loss_ = loss(x,y)
        loss_.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss__.append(loss_.item())

    model.eval()
    with torch.no_grad():
            
        for j, batch in enumerate(test_set):
            cond = 1
            if cond: ### here we do reservoir sampling
                x, y = batch[0].cuda(), batch[1].cuda()
                L.append(sum(torch.argmax(model(x), dim = 1) == y).item())
            else:
                continue
        
    print(t, sum(L)/len(L), sum(loss__)/len(loss__))
    t += 1

0 35.822784810126585 2.7390482486666317
1 37.29113924050633 2.7020462595898174
2 37.55696202531646 2.6640858863625687
3 37.265822784810126 2.6410662347398453
4 37.962025316455694 2.5962708209786576
5 38.0126582278481 2.5644698960092063
6 38.68354430379747 2.531549814107168
7 39.0253164556962 2.5096268275814593
8 38.34177215189873 2.4792110797999154
9 39.50632911392405 2.449424204009268
10 39.45569620253165 2.4229745931942444
11 39.9746835443038 2.395432296616342
12 40.34177215189873 2.36309408501286
13 40.22784810126582 2.343175432871065
14 40.9746835443038 2.318177201558867
15 41.67088607594937 2.299811732738524
16 41.20253164556962 2.2743758463188817
17 41.91139240506329 2.2425808016296545
18 41.75949367088607 2.225042687352661
19 41.40506329113924 2.2017257326399275
20 41.67088607594937 2.1825710242361667
21 41.962025316455694 2.1494564692992384
22 42.48101265822785 2.1371956533178342
23 42.0253164556962 2.10865489570686
24 42.721518987341774 2.0817197881391287
25 42.91139240506329 